In [ ]:
!wget --no-check-certificate "https://www.crcv.ucf.edu/data/UCF101/UCF101.rar"

--2021-07-12 01:14:36--  https://www.crcv.ucf.edu/data/UCF101/UCF101.rar
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6932971618 (6.5G) [application/rar]
Saving to: ‘UCF101.rar’

UCF101.rar          100%[===================>]   6.46G  15.5MB/s    in 7m 11s  

2021-07-12 01:21:48 (15.3 MB/s) - ‘UCF101.rar’ saved [6932971618/6932971618]



In [ ]:
!wget --no-check-certificate "https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip"

--2021-07-12 01:21:48--  https://www.crcv.ucf.edu/data/UCF101/UCF101TrainTestSplits-RecognitionTask.zip
Resolving www.crcv.ucf.edu (www.crcv.ucf.edu)... 132.170.214.127
Connecting to www.crcv.ucf.edu (www.crcv.ucf.edu)|132.170.214.127|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 113943 (111K) [application/zip]
Saving to: ‘UCF101TrainTestSplits-RecognitionTask.zip’

UCF101TrainTestSpli 100%[===================>] 111.27K   295KB/s    in 0.4s    

2021-07-12 01:21:49 (295 KB/s) - ‘UCF101TrainTestSplits-RecognitionTask.zip’ saved [113943/113943]



In [ ]:
!unrar e UCF101.rar Videos/

Streaming output truncated to the last 5000 lines.
Extracting  Videos/v_PlayingFlute_g25_c03.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c04.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c05.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c06.avi                             61%  OK 
Extracting  Videos/v_PlayingFlute_g25_c07.avi                             61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c01.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c02.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c03.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c04.avi                            61%  OK 
Extracting  Videos/v_PlayingGuitar_g01_c05.avi                            61%  OK 
Extracting  Videos/v_PlayingG

In [ ]:
!unzip /content/UCF101TrainTestSplits-RecognitionTask.zip

Archive:  /content/UCF101TrainTestSplits-RecognitionTask.zip
   creating: ucfTrainTestlist/
  inflating: ucfTrainTestlist/classInd.txt  
  inflating: ucfTrainTestlist/testlist01.txt  
  inflating: ucfTrainTestlist/testlist02.txt  
  inflating: ucfTrainTestlist/testlist03.txt  
  inflating: ucfTrainTestlist/trainlist01.txt  
  inflating: ucfTrainTestlist/trainlist02.txt  
  inflating: ucfTrainTestlist/trainlist03.txt  


In [ ]:
import cv2     
import math   
import matplotlib.pyplot as plt    
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   
import numpy as np   
from keras.utils import np_utils
from skimage.transform import resize  
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm


In [ ]:
#  open the .txt file which have names of train videos
f = open("/content/ucfTrainTestlist/trainlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train = train.sample(frac = 0.08)
train.shape

(763, 1)

In [ ]:
train = train.reset_index(drop=True)              # reset the index to the default integer index beginning at 0

In [ ]:
train.head()                        # returning top 5 rows of the train dataframe

,video_name
0,Skiing/v_Skiing_g21_c06.avi 81
1,PlayingDaf/v_PlayingDaf_g25_c07.avi 60
2,PoleVault/v_PoleVault_g23_c06.avi 68
3,TaiChi/v_TaiChi_g14_c01.avi 91
4,Knitting/v_Knitting_g13_c02.avi 50


In [ ]:
# open the .txt file which have names of test videos
f = open("/content/ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test= test.sample(frac=0.015)
test.shape

(57, 1)

In [ ]:
test = test.reset_index(drop=True)

In [ ]:
# creating tags for training videos
train_video_tag = []
for index, row in train.iterrows():
    train_video_tag.append(row['video_name'].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for index, row in test.iterrows():
    test_video_tag.append(row['video_name'].split('/')[0])
    
test['tag'] = test_video_tag

In [ ]:
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('Videos/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5)                                                    #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1)                                                  #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            filename ='train_little/' + videoFile.split('/')[1].split(' ')[0] +"_frame%d.jpg" % count;count+=1     # storing the frames in a new folder named train_little
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 763/763 [01:00<00:00, 12.53it/s]


In [ ]:
# getting the names of all the images
images = glob("train_little/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    train_image.append(images[i].split('/')[1])                           # creating the image name
    train_class.append(images[i].split('/')[1].split('_')[1])             # creating the class of image
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('Videos/train_new.csv',header=True, index=False)

100%|██████████| 5947/5947 [00:00<00:00, 501902.01it/s]


In [ ]:
df = pd.read_csv('/content/Videos/train_new.csv')

In [ ]:
df.shape

(5947, 2)

In [ ]:
# creating an empty list
train_image = []
from keras.preprocessing import image                       

# for loop to read and store frames
for i in tqdm(range(train_data.shape[0])):

    img = image.load_img('train_little/'+train_data['image'][i], target_size=(224,224,3))      # loading the image and keeping the target size as (224,224,3)
    img = image.img_to_array(img)                  # converting it to array
    img = img/255                                  # normalizing the pixel value 
    train_image.append(img)                        # appending the image to the train_image list
    
# converting the list to numpy array
X = np.array(train_image)

# shape of the array
X.shape

100%|██████████| 5947/5947 [00:17<00:00, 334.48it/s]


(5947, 224, 224, 3)

In [ ]:
# separating the target
y = train_data['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [ ]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [ ]:
from keras.applications.vgg16 import VGG16
base_model = VGG16(weights='imagenet', include_top=False)        # creating the base model of pre-trained VGG16 model

58900480/58889256 [==============================] - 2s 0us/step


By the following model, I extracted the features out of the VGG16 network, now I will use these feature as a input to FC layers to get the classification . 


In [ ]:
# extracting features for training frames 
X_train = base_model.predict(X_train)
X_train.shape

(4162, 7, 7, 512)

In [ ]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

(1785, 7, 7, 512)

In [ ]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(4162, 7*7*512)
X_test = X_test.reshape(1785, 7*7*512)

In [ ]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [ ]:
# shape of images
X_train.shape

(4162, 25088)

In [ ]:
y_train.shape

(4162, 101)

In [ ]:
y_test.shape

(1785, 101)

In [ ]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [ ]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [ ]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))          # successive Dense layers to shrink the feature and make feature more and more complex along the way.
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  
model.add(Dense(101, activation='softmax'))        # as ucf101 dataset has 101 classes so, classification length is 101

In [ ]:
# defining a function to save the weights of best model
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)


Epoch 1/100
33/33 [==============================] - 23s 399ms/step - loss: 4.6596 - accuracy: 0.0171 - val_loss: 4.5421 - val_accuracy: 0.0359
Epoch 2/100
33/33 [==============================] - 12s 372ms/step - loss: 4.5287 - accuracy: 0.0257 - val_loss: 4.4408 - val_accuracy: 0.0426
Epoch 3/100
33/33 [==============================] - 12s 372ms/step - loss: 4.4175 - accuracy: 0.0331 - val_loss: 4.1978 - val_accuracy: 0.0796
Epoch 4/100
33/33 [==============================] - 12s 373ms/step - loss: 4.1809 - accuracy: 0.0720 - val_loss: 3.8918 - val_accuracy: 0.1406
Epoch 5/100
33/33 [==============================] - 12s 373ms/step - loss: 3.9264 - accuracy: 0.1172 - val_loss: 3.6207 - val_accuracy: 0.2207
Epoch 6/100
33/33 [==============================] - 12s 376ms/step - loss: 3.6771 - accuracy: 0.1696 - val_loss: 3.3503 - val_accuracy: 0.2695
Epoch 7/100
33/33 [==============================] - 13s 397ms/step - loss: 3.3916 - accuracy: 0.2100 - val_loss: 3.0484 - val_accuracy:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
y_train.shape

In [ ]:
X_train.shape

In [ ]:
f = open("/content/ucfTrainTestlist/testlist01.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()


test['video_name'] = videos
test = test[:-1]
test = test.sample(frac = 0.01)
test= test.reset_index(drop= True)
test_videos = test['video_name']
test.head()

In [ ]:
test_videos.head()

In [ ]:
train = pd.read_csv('Videos/train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [ ]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture('Videos/'+videoFile.split(' ')[0].split('/')[1])   # capturing the video from the given path
    frameRate = cap.get(5)                                                    #frame rate
    x=1
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    while(cap.isOpened()):
        frameId = cap.get(1)                                                  #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            filename ='temp/' + "_frame%d.jpg" % count;count+=1              # storing the frames of this particular video in temp folder
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    
    prediction_images = []
    for p in range(len(images)):
        img = image.load_img(images[p], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
    
    prediction_images3 = np.array(prediction_images)                       # converting all the frames for a test video into numpy array
    print(prediction_images3.shape)                                        # extracting features using pre-trained model

    prediction_images1 = base_model.predict(prediction_images3)            # prediction_images3 = prediction_images3.reshape(())
    prediction_images2 = prediction_images1.reshape(prediction_images1.shape[0], 7*7*512)           # converting features in one dimensional array
    prediction = model.predict_classes(prediction_images2)                 # predicting tags for each array 
    
    predict.append(y.columns.values[s.mode(prediction)[0][0]])             # appending the mode of predictions in predict list to assign the tag to the video
    actual.append(videoFile.split('/')[1].split('_')[1])                   # appending the actual tag of the video

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

We are only getting the accuracy of 29% because we have used 10% of the training data to train our model. 

###why have we done this?
Cause If I was using more than this pecent of data, the RAM was getting exceeded, and we were getting out of the memory constraints of this colab environment. 
